This notebook build predictive models for each municipality

In [1]:
#importer libraries
from sklearn.metrics import mean_squared_error
import os
import tqdm as tqdm
import re
import pickle
from pathlib import Path
import pandas as pd
import numpy as np
# from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.model_selection import GridSearchCV, learning_curve, KFold, train_test_split
# from sklearn.model_selection import learning_curve
# from sklearn.model_selection import KFold, train_test_split

from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso

from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.exceptions import ConvergenceWarning


import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# a function to add features to data.
def add_features_to_data(df):
    fp = Path("../Feature_data/")
    indk = fp/"indkp101.csv"
    konth = fp/"kontanth.csv"
    areas = fp/"muni_areas.csv"
    pop = fp/"population_data.csv"
    pop_dens = fp/"pop_dens.csv"
    gini = fp/"gini_index.csv"
    unenp = fp/"unemployment_data.csv"
        
    indk = pd.read_csv(indk) # ok
    konth = pd.read_csv(konth) #ok men pr kapita
    gini = pd.read_csv(gini) #OK
    areas = pd.read_csv(areas) # ok
    pop = pd.read_csv(pop) # noget galt
    unenp = pd.read_csv(unenp)
    
    df = df\
        .merge(indk, on=["muni_code", "year"], how = 'left',suffixes=('_left', '_right'))\
        .merge(konth, on= ["muni_code", "year"], how= 'left',suffixes=('_left', '_right'))\
        .merge(gini, on = ["muni_code", "year"], how = 'left',suffixes=('_left', '_right'))\
        .merge(areas, on = ["muni_code"], how = 'left',suffixes=('_left', '_right'))\
        .merge(unenp, on = ["muni_code", "year"], how = 'left',suffixes=('_left', '_right'))\
        .merge(pop, on = ["muni_code","year"], how = 'left',suffixes=('_left', '_right'))\
        .sort_values(["year", "count"])\
        .dropna()\
        .assign(muni_code=lambda x: x['muni_code'].astype('category'))\
        .assign(year=lambda x: x['year'].astype('category'))\
        .assign(housing_type = lambda x: x["housing_type"].astype('category'))\
        .assign(unemployed = lambda x: x["unemployed"]/x['pop'])\
        .assign(kont_recip_tot = lambda x: x["kont_recip_tot"]/x['pop'])\
        .assign(pop_den= lambda x: x['pop']/x['km2'])
    
   

    df['avg_sqm_price'] = pd.to_numeric(df['avg_sqm_price'], errors='coerce')

    #drops very useless columns
    cols_to_drop = [col for col in df.columns if col.startswith('Unnamed')]
    df.drop(columns=cols_to_drop, inplace=True)
    df.drop(columns="count", inplace =True)

    return (df)



A function for fitting a model is made. This function takes a dataset of cleaned Boliga data, and enrich it with the selected features.

In [3]:
def make_a_model(data):
    # splitting data in target values (y) and features (X)
    y = data["avg_sqm_price"]
    X = data.drop(columns=["avg_sqm_price"])
    
    # defines scaler for y-data
    y_scaler = StandardScaler()
    y = y_scaler.fit_transform(y.values.reshape(-1, 1)).flatten()
    
    # numeric and categorical features are identified
    numeric_features = X.select_dtypes(include = ["number"]).columns.tolist()
    categorical_features = X.select_dtypes(include=["category"]).columns.tolist()
    # Known categories in the categorical data are identified and stored for use in OneHotEncoder
    known_categories = [X[i].unique().tolist() for i in X.select_dtypes(include=["category"]).columns.tolist()]
    
    # defining transformer for numeric features
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler(with_mean = False)),
        ('poly', PolynomialFeatures(degree=2))
        ])
    
    # Defining transformer for categorical featuresn
    categorical_transformer = OneHotEncoder(categories=known_categories)
    
    # Data is split into test and training data, stratified on housing_type
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=(.2), random_state=47, stratify=X["housing_type"])
    
    
    # Preprocessor defined. Numerical features are scaled, and categorical values OneHotEncoded with the
    # known categories
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

    # The training pipeline is defined. Preprocessing as defined above, polynomial feature expansion
    # and Elastic Net as the classifier
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', ElasticNet())
    ])

    # Paramergrid defined for the gridsearch
    param_grid = {
        'preprocessor__num__poly__degree': [1, 2, 3],
        'classifier__alpha': np.logspace(-4, 4, 12),
        'classifier__l1_ratio': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
        'classifier__max_iter': [2000] 
    }
    # Setting up the GridSearch with pipeline and parametergrid. 5-fold crossvalidation 
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
    
    # Searching for optimal hyperparameters.
    grid_search.fit(X_train, y_train)
    
    # grabbing information about the result
    best_parameters = grid_search.best_params_
    best_pipeline = grid_search.best_estimator_
    
    y_pred = y_scaler.inverse_transform(best_pipeline.predict(X_test).reshape(-1, 1)).flatten()
    y_test_inv = y_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

    
    rmse = mean_squared_error(y_test_inv, y_pred, squared=False)
    r2 = r2_score(y_test_inv, y_pred)
    coefficients = best_pipeline.named_steps['classifier'].coef_

    # Grabbing names and weights of the polynomial features.
    # First, get names of both numeric and categorical features
    numeric_feature_names = numeric_features
    categorical_feature_names = best_pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features)
    all_feature_names = np.concatenate([numeric_feature_names, categorical_feature_names])

    # Now the polynomial feature names
    #polynomial_feature_names = best_pipeline.named_steps['polynomial'].get_feature_names_out(input_features=all_feature_names)
    # ovenstående kan slettes hvis det kører.
    #polynomial_feature_names = best_pipeline.named_steps['preprocessor'].named_transformers_['num'].named_steps['poly'].get_feature_names_out(input_features=all_feature_names)
    
    # For numeriske features
    poly_transformer = best_pipeline.named_steps['preprocessor'].named_transformers_['num'].named_steps['poly']
    numeric_polynomial_feature_names = poly_transformer.get_feature_names_out(input_features=numeric_feature_names)

    # For kategoriske features
    categorical_feature_names = best_pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(input_features=categorical_features)

    # Sammenkæd dem
    polynomial_feature_names = np.concatenate([numeric_polynomial_feature_names, categorical_feature_names])

    
    
    # Combining to one object
    coefs =  zip(coefficients, polynomial_feature_names)
    
    # gets data for a learning curve
    train_sizes, train_scores, test_scores = learning_curve(estimator=best_pipeline,
                   X=X_train,
                   y=y_train,
                   train_sizes=np.arange(0.05, 1.05, .05),
                   scoring='neg_mean_squared_error',                 
                   cv=10)
    
    learning_curve_data = pd.DataFrame({'Train':-train_scores.mean(axis=1),
                     'Test':-test_scores.mean(axis=1),
                     'sample size':train_sizes})
    
    # Finally return fitted models, parameters, metrics, coefficients and data for a learning curve
    return (grid_search, best_parameters, rmse, r2, coefs, learning_curve_data)

    

In [4]:
# Getting aggregated data stored as csv's
fp = Path("../Boliga data/agg_data/")
files = list(fp.glob('*.csv'))

# initialising dataframes for saving results of the fits
metrics = pd.DataFrame(columns=['muni_code', 'rmse', 'r2'])
fitted_models = pd.DataFrame(columns=['muni_code', 'pickled_model'])
learning_curves = pd.DataFrame(columns=['muni_code',  "Train","Test", "sample size"])
coefficients = pd.DataFrame(columns=['muni_code', 'value', 'parameter'])
parameters = pd.DataFrame(columns=['muni_code', 'Parameter', 'Value']) 

data_to_concat = []  # Collect data frames to concatenate

# running the loop for modelling
for filename in tqdm.tqdm(sorted(files)):
    print(filename)
    muni_code = re.search(r'(\d+)\.csv$', str(filename)).group(1)  # extracting muni_code
    data = pd.read_csv(filename)  # reading data
    data = add_features_to_data(data)  # feature adding
    data = data.drop(columns=["year", "muni_code"])  # dropping columns
    grid_search, best_parameters, rmse, r2, coefs, learning_curve_data = make_a_model(data)

    # saving pickled models
    rick = pickle.dumps(grid_search)
    model_row = pd.DataFrame({'muni_code': [muni_code],
                              'pickled_model': [rick]})
    data_to_concat.append(model_row)

    # saving metrics
    metric_tuple = (muni_code, rmse, r2)
    metric_row = pd.DataFrame([metric_tuple], columns=metrics.columns)
    data_to_concat.append(metric_row)

    # saving parameters
    param_row = pd.DataFrame(list(best_parameters.items()), columns=['Parameter', 'Value'])
    param_row['muni_code'] = muni_code
    data_to_concat.append(param_row)

    # saving coefficients
    coef_row = pd.DataFrame(coefs, columns=['value', 'parameter'])
    coef_row['muni_code'] = muni_code
    data_to_concat.append(coef_row)

    # saving learning curve data
    learning_curve_data['muni_code'] = muni_code
    data_to_concat.append(learning_curve_data)

# Concatenate all collected data frames
fitted_models = pd.concat(data_to_concat, ignore_index=True)


  0%|                                                                                           | 0/98 [00:00<?, ?it/s]

..\Boliga data\agg_data\agg_sales_1992_2022_101.csv


  1%|▊                                                                                | 1/98 [01:19<2:07:58, 79.16s/it]

..\Boliga data\agg_data\agg_sales_1992_2022_147.csv


  2%|█▋                                                                               | 2/98 [02:41<2:09:20, 80.84s/it]

...

 98%|██████████████████████████████████████████████████████████████████████████████▎ | 96/98 [2:19:42<02:39, 79.71s/it]

..\Boliga data\agg_data\agg_sales_1992_2022_851.csv


 99%|███████████████████████████████████████████████████████████████████████████████▏| 97/98 [2:20:58<01:18, 78.74s/it]

..\Boliga data\agg_data\agg_sales_1992_2022_860.csv


100%|████████████████████████████████████████████████████████████████████████████████| 98/98 [2:22:17<00:00, 87.12s/it]


In [6]:

# fitted_models 

parameters.to_csv("fitted_models/parameter_elastic.csv")
coefficients.to_csv("fitted_models/coefficients_elastic.csv")
metrics.to_csv("fitted_models/metrics_elastic.csv")
learning_curves.to_csv("fitted_models/learning_curves_elastic.csv")
fitted_models.to_csv("fitted_models/fitted_models_elastic.csv")
parameters.to_csv("fitted_models/parameters_elastic.csv")

In [ ]:
def learning_curve_plot(tester):
    f_learn, ax = plt.subplots(figsize=(7,3))
    ax.plot(tester["sample size"],np.sqrt(tester["Test"]), alpha=0.25, linewidth=2, label ='Test', color='blue') # negated, because we already use neg_MSE
    ax.plot(tester["sample size"],np.sqrt(tester["Train"]), alpha=0.25, linewidth=2, label='Train', color='orange') # negated, because we already use neg_MSE

    ax.set_title('Mean performance')
    ax.set_ylabel('Root-Mean squared error')
    ax.legend();
learning_curve_plot(tester)